# 05: Bagging and random forests

In [139]:
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

import mylib as my

The goal of ensemble methods to to reduce bias and/or variance help prevent overfitting. In this notebook we look at two ensemble methods: bagging and random forests.

## Bootstrap samples
Let's start by seeing how we can draw a bootstrap sample given a dataset $D$. A bootstrap sample is a sample drawn randomly with replacement from the given dataset such that the size of the sample is the same as the size of the original dataset. That means some examples will show up multiple times in the drawn sample.

In the example below, we are using a subset of the car dataset with classes indicating whether the car is in acceptable or unacceptable condition. The description of the original car dataset can be found at [this page](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation).

In [140]:
df = pd.read_csv('datasets/ua_car.csv')
ds = my.DataSet(df, y=True)
print(df.iloc[:,-1].value_counts())

unacc    384
acc      384
Name: y, dtype: int64


In [141]:
train, test = ds.train_test_split(test_portion=.25, shuffle=True)
print(train)
print(test)

    buying maintenance  doors persons luggage safety      y
196  vhigh         low      3       2     med   high  unacc
160   high         low  5more       4     med    low  unacc
479   high        high      4    more     med   high    acc
140    low         low      2       2     med   high  unacc
291  vhigh       vhigh      3    more   small    low  unacc
..     ...         ...    ...     ...     ...    ...    ...
569    med       vhigh      2    more     big    med    acc
296   high         med  5more       2   small    low  unacc
476   high        high      4       4     big   high    acc
637    med         med      2       4   small   high    acc
189  vhigh         low      4       4   small    low  unacc

[576 rows x 7 columns]
    buying maintenance  doors persons luggage safety      y
627    med        high  5more       4     med    med    acc
384  vhigh         med      2       4   small   high    acc
324  vhigh       vhigh      3       2     med   high  unacc
100    low      

Given the above training set, we can draw a bootstrap sample like this:

In [142]:
sample_indexes = np.random.randint(0, train.N, size=train.N)
# print(sample_indexes)
bootstrap_sample = train.examples.iloc[sample_indexes, :]
bootstrap_sample

,buying,maintenance,doors,persons,luggage,safety,y
580,med,vhigh,4,4,small,high,acc
714,low,vhigh,5more,more,big,high,acc
162,vhigh,low,3,2,small,med,unacc
622,med,high,4,more,med,med,acc
712,low,vhigh,5more,more,med,high,acc
...,...,...,...,...,...,...,...
21,vhigh,high,5more,2,med,high,unacc
124,high,vhigh,4,4,big,med,unacc
68,vhigh,high,4,4,small,low,unacc
466,high,high,3,4,big,high,acc


of which:

In [143]:
print("{:.2%}".format(
    pd.unique(bootstrap_sample.index).shape[0] / len(bootstrap_sample)), 'are unique examples')
print("{:.2%}".format(
    1 - pd.unique(bootstrap_sample.index).shape[0] / len(bootstrap_sample)), 'are repeated examples')

62.15% are unique examples
37.85% are repeated examples


Sometimes, it's useful to be able to identify the examples that are included in a given sample and those that aren't. Here are two functions for doing so.

In [144]:
def examples_in_sample(examples, sample):
    return examples[examples.index.isin(sample.index)]

# can i just turn this into the in bag or out of bag?
def examples_not_in_sample(examples, sample):
    return examples[~examples.index.isin(sample.index)]

Here are the examples from the training set what are in the above bootstrap sample:

In [145]:
examples_in_sample(train.examples, bootstrap_sample)

,buying,maintenance,doors,persons,luggage,safety,y
479,high,high,4,more,med,high,acc
140,low,low,2,2,med,high,unacc
352,low,med,5more,2,med,low,unacc
695,low,vhigh,4,4,small,high,acc
707,low,vhigh,5more,4,med,high,acc
...,...,...,...,...,...,...,...
569,med,vhigh,2,more,big,med,acc
296,high,med,5more,2,small,low,unacc
476,high,high,4,4,big,high,acc
637,med,med,2,4,small,high,acc


And here are the examples from the training set that are not in the above bootstrap sample:

In [146]:
type(examples_not_in_sample(train.examples, bootstrap_sample))

pandas.core.frame.DataFrame

## Bagging
The simplest form of ensemble methods is called **bagging** which stands for **bootstrap aggregation**. The idea is simple:
* take $T$ bootstrap samples from the given dataset
* for each bootstrap sample, train a decision tree DT
* the predicted label of an unseen example is the average(for regression problems) or the plurality vote (for classification problems) of all the output predicted by all the trained $T$ trees.

Here is a simple implementation of bagging.

In [147]:
class Bagger:
    def __init__(self, dataset, nTrees):
        self.ds = dataset
        self.nTrees = nTrees
        self.classifiers = []
        self.samples = []
        self.make_trees()

    def make_trees(self):
        indexes = np.random.randint(0, self.ds.N,(self.ds.N,self.nTrees))
        for i in range(self.nTrees):
            # Create bootstrap samples one for each tree
            self.samples.append(self.ds.examples.iloc[indexes[:, i], :])

            # Build classifiers
            self.classifiers.append(my.DecisionTreeClassifier(my.DataSet(self.samples[i])))

    def predict(self, unseen):
        """
        Returns the most probable label (or class) for each unseen input. The
        unseen needs to be a data series with the same features (as indexes) as the 
        training data. It can also be a data frame with the same features as 
        the training data.
        """
        if unseen.ndim == 1:
            classes = np.array([ dt.predict(unseen) for dt in self.classifiers ])
            classes = classes[classes != None]
            return st.mode(classes).mode[0]
        
        else:
            return np.array([self.predict(unseen.iloc[i,:]) for i in range(len(unseen))]) 

## Random forests
Bagging is not exclusive to decision trees; it can be used with other models. Random forests is bagging applied exclusively to decision trees. In addition to obtaining $T$ random bootstrap samples, it also requires what is sometimes called **feature bagging**. Feature bagging requires that only a randomly selected subset of the features is considered at each node during the construction of the decision tree. 

That means we need to modify our implementation of the decision tree such that it takes a numeric parameter named `nFeatures` which defaults to 0. If `nFeatures` is 0, then the tree functions as normal. If not, it picks this many features randomly and only consider the best of those during the construction of the tree. The provided `my.DecisionTreeClassifier` class already has these changes.

For prediction, a plurality vote of the $T$ predicted labels is returned. Here is a simple implementing of random forests. Think about the similarities and differences between these too classes.

In [162]:
class RandomForest:
    def __init__(self, dataset, nTrees, nFeatures=0):
        self.ds = dataset
        self.nTrees = nTrees
        self.nFeatures = nFeatures
        self.classifiers = []
        self.samples = []
        self.make_forest()

    def make_forest(self):
        indexes = np.random.randint(0, self.ds.N,(self.ds.N,self.nTrees))
        for i in range(self.nTrees):
            # Create bootstrap samples one for each tree
            self.samples.append(self.ds.examples.iloc[indexes[:, i], :])

            # Build classifiers
            self.classifiers.append(my.DecisionTreeClassifier(my.DataSet(self.samples[i]), nFeatures=self.nFeatures))

    def predict(self, unseen):
        """
        Returns the most probable label (or class) for each unseen input. The
        unseen needs to be a data series with the same features (as indexes) as the 
        training data. It can also be a data frame with the same features as 
        the training data.
        """
        if unseen.ndim == 1:
            classes = np.array([ dt.predict(unseen) for dt in self.classifiers ])
            classes = classes[classes != None]
            return st.mode(classes,keepdims=True).mode[0]
        
        else:
            return np.array([self.predict(unseen.iloc[i,:]) for i in range(len(unseen))]) 

## Testing

In [149]:
dt = my.DecisionTreeClassifier(train)
cm = my.confusion_matrix(test.target, dt.predict(test.examples.iloc[:,:-1]))
accuracy = np.trace(cm) / np.sum(cm)
# print(test.examples.iloc[1,:])
print(cm)
print('Decistion tree accuracy: ', accuracy)


bg = Bagger(train, 20)
cm = my.confusion_matrix(test.target, bg.predict(test.examples.iloc[:,:-1]))
accuracy = np.trace(cm) / np.sum(cm)

print(cm)
print('Bagger accuracy: ', accuracy)

rf = RandomForest(train, 20, nFeatures=3)
cm = my.confusion_matrix(test.target, rf.predict(test.examples.iloc[:,:-1]))
accuracy = np.trace(cm) / np.sum(cm)

print(cm)
print('Random forests accuracy: ', accuracy)

# print(rf.classifiers)


[[89  2]
 [ 4 97]]
Decistion tree accuracy:  0.96875


C:\Users\graem\AppData\Local\Temp\ipykernel_5408\1636490916.py:28: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return st.mode(classes).mode[0]
c:\Users\graem\anaconda3\envs\ml\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\graem\AppData\Local\Temp\ipykernel_5408\1636490916.py:28: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https:/

[[89  2]
 [11 90]]
Bagger accuracy:  0.9322916666666666
[[91  0]
 [13 88]]
Random forests accuracy:  0.9322916666666666


C:\Users\graem\AppData\Local\Temp\ipykernel_5408\1147172297.py:29: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return st.mode(classes).mode[0]
c:\Users\graem\anaconda3\envs\ml\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\graem\AppData\Local\Temp\ipykernel_5408\1147172297.py:29: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https:/

You should try different values for `nTrees` and `nFeatures`. These variables are considered hyperparameters, and cross-validation can be used to determine the best values for them. Common values for `nFeatures` are $\sqrt{m}$ and $log_2(m)$ where $m$ is the number of features.

## Out of bag score
Another way of testing random forests is to calculate the so-called **out-of-bag** score. Such a score does not require splitting the dataset into a training and test sets. One way to calculate it is to identify for each example $x$ in the dataset the list of trees that are trained using samples that do not include it; let's call this list of trees $D_x$. We then call the `predict` method on each tree of $D_x$ to get the list of predicted classes for each of of these out of bag $x$ examples; let's call this list of classes $C_x$. Finally we find the class in $C_x$ that repeats the most and report it as the predicted class of $x$; let's call it $h_x$.

Doing this for each example in the dataset gives us an array of predicted classes, which we can compare against the actual target classes of these examples. Using the confusion matrix we can report the accuracy as the out of bag score.

Notice that the above implementations of `Bagger` and `RandomForest` already give you access to the bootstrap samples and the classifiers that are trained on them. You can use that to find out what sample does not include a given example.

## CHALLENGE
Write a function that calculates the out of bag score as described above given three arguments: a dataset, number of trees (`nTrees`), and number of features (`nFeatures`). The function should use these arguments to create a random forest object to use for calculating this score.

Test and report the out of bag scores for the whole car dataset and for when `nTrees` is 10, 15, and 20.

In [163]:
def out_of_bag_score(data, nTrees, nFeatures):

    treesNotTrainedWithList = []
    # create the random forest object using in bag dataset
    rf = RandomForest(data, nTrees, nFeatures)

    # for each example in the dataset go through every classifier to see if it was trained or not by it
    for i in range(rf.ds.N):

        out_of_bag_arr = np.zeros(nTrees) # create an array for each example for trees that it is not in
        # go through each list of indices used
        for j in range(nTrees):
            
            # if index is not in the samples used index then it is marked as a 1
            not_in_df = examples_not_in_sample(rf.ds.examples, rf.samples[j])

            if i in not_in_df.index:
                out_of_bag_arr[j] = 1
        # put example out of bag array in not trained list  
        treesNotTrainedWithList.append(out_of_bag_arr) 
    
    # predict the out of bag values using their same classifier
    out_of_bag_mode_list = []

    # go through each predicted sample
    for k in range(rf.ds.N):

        # call predict on each value 
        out_of_bag_preds_list = []
        for l in range(nTrees):
            
            # print(treesNotTrainedWithList[k][l])
            if treesNotTrainedWithList[k][l] == 1:
                # predict using the lth tree if it was an out of bag value
                predVal = rf.classifiers[l].predict(data.examples.iloc[k,:-1])
                out_of_bag_preds_list.append(predVal)
  
        # append the mode of the predicted values 
        if len(out_of_bag_preds_list) > 0:
            # it doesn't like this 
            pred_mode = st.mode(out_of_bag_preds_list,keepdims=True).mode[0]
           

        # if the value was always in the sample just use what the full model would've predicted
        else:
            pred_mode = rf.predict(data.examples.iloc[k,:-1])

        out_of_bag_mode_list.append(pred_mode)

    preds = np.array(out_of_bag_mode_list)         
    cm = my.confusion_matrix(data.target, preds)

    return cm


Testing out ntrees = 10, 15, 20

In [164]:
score10_cm = out_of_bag_score(ds,10,4)
accuracy10 = np.trace(score10_cm) / np.sum(score10_cm)

print(score10_cm)
print('Out of bag score for nTrees = 10: ', accuracy10)

score15_cm = out_of_bag_score(ds,15,4)
accuracy15 = np.trace(score15_cm) / np.sum(score15_cm)

print(score15_cm)
print('Out of bag score for nTrees = 15: ', accuracy15)

score20_cm = out_of_bag_score(ds,20,4)
accuracy20 = np.trace(score20_cm) / np.sum(score20_cm)

print(score20_cm)
print('Out of bag score for nTrees = 20: ', accuracy20)

c:\Users\graem\anaconda3\envs\ml\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\graem\AppData\Local\Temp\ipykernel_5408\3132053637.py:41: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  pred_mode = st.mode(out_of_bag_preds_list,keepdims=True).mode[0]
C:\Users\graem\AppData\Local\Temp\ipykernel_5408\417025228.py:29: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  return st.mode(classes,keepdims=True).mode[0]


[[370  14]
 [ 33 351]]
Out of bag score for nTrees = 10:  0.9388020833333334


c:\Users\graem\anaconda3\envs\ml\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\graem\AppData\Local\Temp\ipykernel_5408\3132053637.py:41: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  pred_mode = st.mode(out_of_bag_preds_list,keepdims=True).mode[0]


[[378   6]
 [ 32 352]]
Out of bag score for nTrees = 15:  0.9505208333333334


c:\Users\graem\anaconda3\envs\ml\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\graem\AppData\Local\Temp\ipykernel_5408\3132053637.py:41: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  pred_mode = st.mode(out_of_bag_preds_list,keepdims=True).mode[0]


[[376   8]
 [ 34 350]]
Out of bag score for nTrees = 20:  0.9453125
